<a href="https://colab.research.google.com/github/lakshaygola/TensorFlow-And-DeepLearning/blob/main/Hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 3.0MB/s 


In [2]:
import keras
import tensorflow as tf
import kerastuner as kt 
from tensorflow import keras

# Downloading the data

Donwnloading the dataset (fashion clothing dataset present in keras) and making the train and test datasets from the data


In [3]:
(image_train, label_train), (image_test, label_test)= keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
label_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [5]:
# Normalizing the image data
image_train= image_train.astype('float') / 255.0
image_test= image_test.astype('float') / 255.0

In [15]:
# Defing the model 
def build_model(hp):
  # Initializing the model
  model= keras.Sequential()
  model.add(keras.layers.Flatten(input_shape= (28,28)))
  
  # To select the numbers of input you want from this layer (that will be between 32, 512)
  random_units= hp.Int('num_layer', 32, 512)
  model.add(keras.layers.Dense(units= random_units, activation= 'relu'))
  # Final output for the layer will be equal to 10
  model.add(keras.layers.Dense(10))

  # Selecting the learning_rate from the given list of numbers
  learning_rate= hp.Choice('learning_rate', values= [0.001, 0.0001, 0.00001])

  # Compiling the model 
  model.compile(optimizer= keras.optimizers.Adam(learning_rate= learning_rate),
                loss= keras.losses.SparseCategoricalCrossentropy(from_logits= True),
                metrics= ['accuracy'])
  
  return model

In [16]:
# Intializing the hyperparameter 
tuner = kt.Hyperband(build_model, 
                     objective= 'val_accuracy',    # which we want to increase in our model
                     max_epochs= 10, 
                     factor= 3,
                     directory= 'my_dir', 
                     project_name= 'intro_to_kit')

In [17]:
# now making the callback for the early stop
early_stop= keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 4)

In [19]:
# Let search the optimial solution for the model
tuner.search(image_train, label_train, epochs=30, validation_split= 0.2, callbacks= [early_stop])

best_param= tuner.get_best_hyperparameters(num_trials= 1)[0]

Trial 31 Complete [00h 00m 42s]
val_accuracy: 0.8320000171661377

Best val_accuracy So Far: 0.8862500190734863
Total elapsed time: 00h 11m 28s
INFO:tensorflow:Oracle triggered exit


In [28]:
best_param.get('learning_rate'), best_param.get('num_layer')

(0.001, 284)